In [ ]:
import os
import sys
import xarray as xr
import cftime
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import time as t_util
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
import matplotlib.colors as colors
import mplotutils as mpu


## Define folders

In [ ]:
dir_SLAND    = '/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_forest   = '/Data/forest_masks/'
dir_ELUC     = '/Data/ELUC_countries/'
dir_peat     = '/Data/Peat_data/'
dir_data_ctr = '/Data/data_ELUC_NGHGI_SLAND_plot/'
dir_LSM      = '/Data/LSMs_Trendy-v10/'
dir_fig      = '/Figures/'


## Read forest map

In [ ]:
#Select year for forest mask
year = '2013'

#Read forest mask
fname_mask = dir_forest +  'Hansen2010_IFL_' + year + '.nc'
for_mask = xr.open_dataset(fname_mask)

#Read forest fraction
fname_Hansen = dir_forest + 'ForestFraction_0.5deg_2013_regrid360x720-ForestMask.nc'
data_Hansen = xr.open_dataset(fname_Hansen)

#Re-index data
check_lat = np.max(np.abs(data_Hansen['lat'].values - for_mask.lat.values))
check_lon = np.max(np.abs(data_Hansen['lon'].values - for_mask.lon.values))
if check_lat>0.01 or check_lon>0.01:  sys.exit('Latitudes do not agree')
data_Hansen = data_Hansen.reindex({'lat': for_mask['lat'], 'lon': for_mask['lon']}, method='nearest')

#Get intact and non-intact forest mask
for_nonintact = data_Hansen.forest_fraction.where(for_mask.Band1!=2)
for_nonintact = for_nonintact > 0.0
for_nonintact = for_nonintact.where(for_nonintact)
for_intact = data_Hansen.forest_fraction.where(for_mask.Band1==2)
for_intact = for_intact > 0.0
for_intact = for_intact.where(for_intact)

#Get intact and non-intact forest mask (for plotting use a 20% threshold for forest cover)
for_nonintact_plot = data_Hansen.forest_fraction.where(for_mask.Band1!=2)
for_nonintact_plot = for_nonintact_plot > 0.2
for_nonintact_plot = for_nonintact_plot.where(for_nonintact_plot)
for_intact_plot = data_Hansen.forest_fraction.where(for_mask.Band1==2)
for_intact_plot = for_intact_plot > 0.2
for_intact_plot = for_intact_plot.where(for_intact_plot)


## Read SLAND data (new)

In [ ]:
time_sta = '2001'
time_end = '2015'

for_limit = 0.20

#Select grid size
# gridsize = '360x720'
gridsize = '360x720-ForestMask'

#Read SLAND data
fname_SLAND = dir_SLAND + "SLAND-weighted_ForestFraction_DGVMs-S2-S3_all-models_" + time_sta + "-" + time_end + "-mean_regrid" + gridsize + ".nc"
data_SLAND  = xr.open_dataset(fname_SLAND)

#Read land-sea mak
fname_LSM = dir_LSM + 'LandSeaMask_' + gridsize + '.nc'
data_LSM  = xr.open_dataset(fname_LSM)

#Calculate multi-model median
data_SLAND_map = data_SLAND.median(dim='model')

#Apply land-sea mask
data_SLAND_map = data_SLAND_map.where(data_LSM.sftlf>=0.5)

#Set correct sign vor SLAND
data_SLAND_map = -data_SLAND_map

#Convert unit from t C / ha / year to t CO2 / ha / year
data_SLAND_map = 44/12 * data_SLAND_map


## Read ELUC data

In [ ]:
version = ''

time_sta = '2001'
time_end = '2015'

#Read data
fname_BLUE_sinks    = dir_ELUC + 'ELUC_BLUE_GCB2021_ELUC-sinks-density_2000-2020.nc'
fname_BLUE_sources  = dir_ELUC + 'ELUC_BLUE_GCB2021_ELUC-sources-density_2000-2020.nc'
fname_HN21_sinks    = dir_ELUC + 'ELUC_H&N_GCB2021_ELUC-sinks-density_2000-2020' + version + '.nc'
fname_HN21_sources  = dir_ELUC + 'ELUC_H&N_GCB2021_ELUC-sources-density_2000-2020' + version + '.nc'
fname_OSCAR_sinks   = dir_ELUC + 'ELUC_OSCAR_GCB2021_ELUC-sinks-density_2000-2020' + version + '.nc'
fname_OSCAR_sources = dir_ELUC + 'ELUC_OSCAR_GCB2021_ELUC-sources-density_2000-2020' + version + '.nc'
data_BLUE_snk  = xr.open_dataset(fname_BLUE_sinks)
data_BLUE_src  = xr.open_dataset(fname_BLUE_sources)
data_HN21_snk  = xr.open_dataset(fname_HN21_sinks)
data_HN21_src  = xr.open_dataset(fname_HN21_sources)
data_OSCAR_snk = xr.open_dataset(fname_OSCAR_sinks)
data_OSCAR_src = xr.open_dataset(fname_OSCAR_sources)

#Read peat data
fname_peat = dir_peat + 'Peat_emissions_2000-2020.nc'
data_peat  = xr.open_dataset(fname_peat)
data_peat  = data_peat.rename({'E_peat': 'ELUC'})
data_peat  = data_peat / 100 # Convert g C/m2 to t C/ha

#Calculate ELUC net
data_BLUE_net  = data_BLUE_snk + data_BLUE_src
data_HN21_net  = data_HN21_snk + data_HN21_src
data_OSCAR_net = data_OSCAR_snk + data_OSCAR_src

#Calculate average over 2001-2015
data_BLUE_net = data_BLUE_net.sel(time=slice(time_sta, time_end)).mean('time')
if version!='_vMean':
    data_HN21_net  = data_HN21_net.sel(time=slice(time_sta, time_end)).mean('time')
    data_OSCAR_net = data_OSCAR_net.sel(time=slice(time_sta, time_end)).mean('time')
data_peat      = data_peat.sel(time=slice(time_sta, time_end)).mean('time')

#Concatenate models and calculate average over models
data_ELUC_map = data_BLUE_net.expand_dims('model')
data_ELUC_map = xr.concat((data_ELUC_map, data_HN21_net, data_OSCAR_net), dim='model')
data_ELUC_map = data_ELUC_map.mean('model')


#Add peat to ELUC
data_ELUC_map = data_ELUC_map + data_peat

#Read land-sea mak
fname_LSM = dir_LSM + 'LandSeaMask_720x1440.nc'
data_LSM  = xr.open_dataset(fname_LSM)

#Apply land-sea mask
data_ELUC_map = data_ELUC_map.where(data_LSM.sftlf>=0.05)

#Convert unit from t C/ha to t CO2/ha
data_ELUC_map = 44/12 * data_ELUC_map


## Read SLAND on country level

In [ ]:
time_sta = '2001'
time_end = '2015'

#Read SLAND
file_name  = dir_data_ctr + 'Collection_SLAND_total-weighted_' + time_sta + '-' + time_end + '.pickle'
data_SLAND = pd.read_pickle(file_name)

#Read SLAND on forests
file_name = dir_data_ctr + 'Collection_SLAND_non-intact-forest_mask' + year + '_' + time_sta + '-' + time_end + '.pickle'
data_SLAND_forest = pd.read_pickle(file_name)

#Convert unit from Tg C/year to Pg CO2/year
data_SLAND        = 44/12 / 1000 * data_SLAND
data_SLAND_forest = 44/12 / 1000 * data_SLAND_forest

#Select SLAND median for specific countries
countries_sel = ['BRA', 'CAN', 'CHN', 'COD', 'IDN', 'RUS', 'USA', 'EU27_UK']
data_SLAND_all_sel = data_SLAND.loc[countries_sel].median(axis=1)
data_SLAND_for_sel = data_SLAND_forest.loc[countries_sel].median(axis=1)

#Get uncertainty
models_SLAND = data_SLAND_forest.columns
data_SLAND_all_unc = data_SLAND.loc[countries_sel][models_SLAND]
data_SLAND_for_unc = data_SLAND_forest.loc[countries_sel][models_SLAND]

#Save uncertainty in dataframe
data_SLAND_unc = pd.DataFrame()
data_SLAND_unc['q25_all'] = data_SLAND_all_unc.quantile(0.25, axis=1)
data_SLAND_unc['q75_all'] = data_SLAND_all_unc.quantile(0.75, axis=1)
data_SLAND_unc['q25_for'] = data_SLAND_for_unc.quantile(0.25, axis=1)
data_SLAND_unc['q75_for'] = data_SLAND_for_unc.quantile(0.75, axis=1)


## Plot preparation

In [ ]:
col_SLAND_for = '#01665e'
col_SLAND_all = '#80cdc1'
col_ELUC      = 'sienna'
col_intact    = 'k'
col_nonintact = 'k'

vmin = -2.7
vmax = 2.7

#Calculate colors for uncertainty
fraction = 0.6
col1 = np.array(colors.to_rgba(col_SLAND_for)[0:3]) * 255
col2 = np.array([0, 0, 0])
col_SLAND_for_2 = ( (col1 - col2) * fraction + col2 ) / 255

#Calculate colors for uncertainty
fraction = 0.6
col1 = np.array(colors.to_rgba(col_SLAND_all)[0:3]) * 255
col2 = np.array([0, 0, 0])
col_SLAND_all_2 = ( (col1 - col2) * fraction + col2 ) / 255

#Define country list for EU27 + UK (excluding Malta because it is not in shape file)
ctrs_EU = ['AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'NLD', 'POL', 'PR1', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'GBR'] #'MLT', 

#Read shapes of countries on Earth
shpfilename = shpreader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)


## Plot

In [ ]:
#Select whether to include bars for countries or not
include_country_bars = 1

#Set hatching linewidth
mpl.rcParams['hatch.linewidth'] = 0.75

#Define hatching colors and patterns
col_intact      = 'chocolate'
col_nonintact   = 'sienna'
hatch_nonintact = 'xxxx'
hatch_intact    = '////'

#Create figure
fig, axes = plt.subplots(2, 1, figsize=(12, 13), subplot_kw=dict(projection=ccrs.Robinson()))
plt.subplots_adjust(hspace=0.2)



#### ELUC ####
    
ax = axes[0]
    
#Coastlines and borders
ax.coastlines(linewidth=0.5, color='#878787')
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='#878787')

#Plot map
h1 = ax.pcolormesh(data_ELUC_map['lon'], data_ELUC_map['lat'], data_ELUC_map['ELUC'], transform=ccrs.PlateCarree(), shading='auto', cmap='BrBG_r', vmin=vmin, vmax=vmax, rasterized=True)

#Set extent to global
ax.set_global()

#Title
ax.set_title('Anthropogenic LULUCF flux', fontsize=22, fontweight='bold', pad=15)

#Plot country borders
for ctr in countries_sel:
    if ctr=='EU27_UK':  continue
    country_sel = [country for country in reader.records() if country.attributes["SU_A3"] == ctr][0]
    shape_feature = ShapelyFeature([country_sel.geometry], ccrs.PlateCarree(), facecolor="none", linestyle='-', edgecolor='k', lw=0.75)
    ax.add_feature(shape_feature)
    
#Get border for EU27 & GB
create = 1
for ctr in ctrs_EU:
    country_sel = [country for country in reader.records() if country.attributes["SU_A3"] == ctr][0]
    if create==1:
        EU_shape = country_sel.geometry
        create = 0
    else:
        EU_shape = EU_shape.union(country_sel.geometry)
        
#Plot border for EU27 & GB
shape_feature = ShapelyFeature([EU_shape], ccrs.PlateCarree(), facecolor="none", linestyle='-', edgecolor='k', lw=0.75)
ax.add_feature(shape_feature)    

#Add 'a' and 'b'
plt.text(0.05, 1.02, 'A', fontweight='bold', fontsize=22, transform=axes[0].transAxes)
plt.text(0.05, 1.02, 'B', fontweight='bold', fontsize=22, transform=axes[1].transAxes)



#### SLAND ####

ax = axes[1]

#Coastlines and borders
ax.coastlines(linewidth=0.5, color='#878787')
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='#878787')

#Plot map
h1 = ax.pcolormesh(data_SLAND_map['lon'], data_SLAND_map['lat'], data_SLAND_map.SLAND, transform=ccrs.PlateCarree(), shading='auto', cmap='BrBG_r', vmin=vmin, vmax=vmax, rasterized=True)
h2 = ax.contourf(for_nonintact_plot.lon, for_nonintact_plot.lat, for_nonintact_plot, hatches=[hatch_nonintact], transform=ccrs.PlateCarree())
h3 = ax.contourf(for_intact_plot.lon, for_intact_plot.lat, for_intact_plot, hatches=[hatch_intact], transform=ccrs.PlateCarree())

#Color hatches
for collection in h2.collections:  collection.set_edgecolor(col_nonintact)
for collection in h2.collections:  collection.set_linewidth(0.)
for collection in h2.collections:  collection.set_facecolor('none')
for collection in h3.collections:  collection.set_edgecolor(col_intact)
for collection in h3.collections:  collection.set_linewidth(0.)
for collection in h3.collections:  collection.set_facecolor('none')

#Title
ax.set_title('Natural land flux', fontsize=22, fontweight='bold', pad=15)

#Set extent to global
ax.set_global()

#Loop over countires
for ctr in countries_sel:

    #Extract data
    SLAND_all = data_SLAND_all_sel[ctr]
    SLAND_for = data_SLAND_for_sel[ctr]
    SLAND_all_q25 = data_SLAND_unc['q25_all'][ctr]
    SLAND_all_q75 = data_SLAND_unc['q75_all'][ctr]
    SLAND_for_q25 = data_SLAND_unc['q25_for'][ctr]
    SLAND_for_q75 = data_SLAND_unc['q75_for'][ctr]

    #Set position of axis
    if ctr=='BRA':        ax1 = fig.add_axes([0.428, 0.143, 0.02, 0.075])
    elif ctr=='CHN':      ax1 = fig.add_axes([0.768, 0.274, 0.02, 0.075])
    elif ctr=='IDN':      ax1 = fig.add_axes([0.695, 0.195, 0.02, 0.075])
    elif ctr=='COD':      ax1 = fig.add_axes([0.500, 0.213, 0.02, 0.075])
    elif ctr=='RUS':      ax1 = fig.add_axes([0.805, 0.36, 0.02, 0.075])
    elif ctr=='USA':      ax1 = fig.add_axes([0.385, 0.293, 0.02, 0.075])
    elif ctr=='CAN':      ax1 = fig.add_axes([0.250, 0.322, 0.02, 0.075])
    elif ctr=='EU27_UK':  ax1 = fig.add_axes([0.467, 0.332, 0.02, 0.075])
    else:             sys.exit('Country location for ' + ctr + ' not defined!')

    #Remove background color and frame
    ax1.set_facecolor('none')
    ax1.axis("off")

    txt_out = '_no-bars'
    if include_country_bars==1:

        txt_out = ''

        #Plot bars
        ax1.bar(0.5, SLAND_for, width=1, color=col_SLAND_for, zorder=9)
        ax1.bar(1.5, SLAND_all, width=1, color=col_SLAND_all, zorder=9)
    
        #Plot uncertainty
        ax1.plot([0.5, 0.5], [SLAND_for_q25, SLAND_for_q75], color='k', zorder=11)
        ax1.plot([1.5, 1.5], [SLAND_all_q25, SLAND_all_q75], color='k', zorder=11)

        #Limits
        ax1.set_xlim([0, 2])
        ax1.set_ylim([-2.16, 0])
        ylim = ax1.get_ylim()

        #Text
        y_txt1 = ylim[0] + 0.8 * np.diff(ylim)
        y_txt2 = ylim[0] + 0.6 * np.diff(ylim)
        if ctr in ['CAN', 'COD', 'IDN', 'EU27_UK']:
            x_txt = -0.2
            ha = 'right'
        else:
            x_txt = 2.2
            ha = 'left'
        if ctr=='BRA':
            add_text1 = ' (managed forest, Pg CO$_\mathbf{2}$/yr)'
            add_text2 = ' (all land, Pg CO$_\mathbf{2}$/yr)'
            ax.add_patch(mpl.patches.Rectangle((0.65, 0.17), 0.05, 0.05, facecolor='w', transform=ax.transAxes, zorder=5))
        else:
            add_text1 = ''
            add_text2 = ''
        ax1.text(x_txt, y_txt1, '{:.2f}'.format(SLAND_for) + add_text1, fontsize=11, ha=ha, fontweight='bold', color=col_SLAND_for, zorder=7)
        ax1.text(x_txt, y_txt2, '{:.2f}'.format(SLAND_all) + add_text2, fontsize=11, ha=ha, fontweight='bold', color=col_SLAND_all, zorder=7)

    #Title
    if ctr=='EU27_UK':   ax1.set_title('EU&UK', x=-0.1, pad=-5)
    else:                ax1.set_title(ctr, pad=-5)

#Plot country borders
for ctr in countries_sel:
    if ctr=='EU27_UK': continue
    country_sel = [country for country in reader.records() if country.attributes["SU_A3"] == ctr][0]
    shape_feature = ShapelyFeature([country_sel.geometry], ccrs.PlateCarree(), facecolor="none", linestyle='-', edgecolor='k', lw=0.75)
    ax.add_feature(shape_feature)
    
#Get border for EU27 & GB
create = 1
for ctr in ctrs_EU:
    country_sel = [country for country in reader.records() if country.attributes["SU_A3"] == ctr][0]
    if create==1:
        EU_shape = country_sel.geometry
        create = 0
    else:
        EU_shape = EU_shape.union(country_sel.geometry)
        
#Plot border for EU27 & GB
shape_feature = ShapelyFeature([EU_shape], ccrs.PlateCarree(), facecolor="none", linestyle='-', edgecolor='k', lw=0.75)
ax.add_feature(shape_feature)    

#Create legend (using dummy scatter plots)
sc2 = ax.scatter(3, 3, s=20**2, marker='s', facecolor='white', linewidth=0, edgecolor=col_nonintact, hatch=hatch_nonintact, transform=fig.transFigure)
sc1 = ax.scatter(3, 3, s=20**2, marker='s', facecolor='white', linewidth=0, edgecolor=col_intact, hatch=hatch_intact, transform=fig.transFigure)
ax.legend([sc2, sc1], ['managed forest', 'unmanaged forest'], loc='center', bbox_to_anchor=(0.16, 0.4), fontsize=14, frameon=False, labelspacing=0.75, handletextpad=0.4)

### Colorbar and save figure ###

#Colorbar
cbar = mpu.colorbar(h1, axes[1], orientation='horizontal', extend='both', size=0.07, shrink=0.2, pad=0.12)
cbar.ax.tick_params(labelsize=16)
cbar.set_label('CO$_2$ flux (t CO$_2$ / ha / yr)', fontsize=20, labelpad=20)

#Add text to colorbar
ax.text(0.18, -0.29, 'sinks', fontsize=18, ha = 'right', transform=ax.transAxes)
ax.text(0.82, -0.29, 'emissions', fontsize=18, ha = 'left', transform=ax.transAxes)

#Save figure
fig.savefig(dir_fig + 'Fig1_SLAND_ELUC_with_peat' + txt_out + version + '.png', dpi=300, bbox_inches='tight')
fig.savefig(dir_fig + 'Fig1_SLAND_ELUC_with_peat' + txt_out + version + '.pdf', bbox_inches='tight')
